In [1]:
# !pip install arxiv
# !pip install wikipedia

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
wiki.name

'wikipedia'

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://python.langchain.com/v0.1/docs/modules/tools/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vector_db = FAISS.from_documents(documents, OllamaEmbeddings())
retriever = vector_db.as_retriever()
retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x744a603f9270>)

In [6]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about Langchain. For any questions about Langchain, you must use this tool!")

In [7]:
retriever_tool.name

'langsmith_search'

In [8]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=5, doc_content_chars_max=500)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)


In [9]:
tools = [wiki, arxiv_tool, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/anas/Documents/Learn-Langchain/venv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=5, lang='en', load_all_available_meta=False, doc_content_chars_max=500)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=5, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=500)),
 Tool(name='langsmith_search', description='Search for information about Langchain. For any questions about Langchain, you must use this tool!', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x744a74f41870>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community

In [10]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [11]:
# !pip install langchainhub

In [12]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [13]:
### Agents

from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [14]:
### Agent Executer
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [15]:
agent_executor.invoke({"input" : "Tell me about Langchain"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langchain'}`


Tools | 🦜️🔗 LangChain

of tools that work well together. For a more in depth description as well as a list of all built-in toolkits, see this pageTools as OpenAI Functions: Tools are very similar to OpenAI Functions, and can easily be converted to that format. See this notebook for instructions on how to do that.Help us out by providing feedback on this documentation page:PreviousCompositionNextToolkitsDefault ToolsCustomizing Default ToolsMore TopicsCommunityDiscordTwitterGitHubPythonJS/TSMoreHomepageBlogYouTubeCopyright © 2024 LangChain, Inc.

Skip to main contentLangChain v0.2 is out! You are currently viewing the old v0.1 docs. View the latest docs here.ComponentsIntegrationsGuidesAPI ReferenceMorePeopleVersioningContributingTemplatesCookbooksTutorialsYouTubev0.1v0.2v0.1🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchModel I/OPro

{'input': 'Tell me about Langchain',
 'output': 'Langchain is a tool that provides interfaces for agents, chains, or LLMs to interact with the world. It combines the name of the tool, a description of what the tool is, JSON schema of the inputs to the tool, and the function to call. Tools in Langchain are designed to prompt the LLM on how to specify actions and take those actions effectively. The simpler the input to a tool, the easier it is for an LLM to utilize it.'}

In [16]:
agent_executor.invoke({"input" : "I want to learn about Machine Learnin"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions. Recently, artificial neural networks have been able to surpass many previous approaches in performance.
ML finds application in many fields, including natural language processing, computer vision, speech recognition, email Machine learning (ML) is a field of study in artificial intelligence that focuses on the development and study of statistical algorithms. These algorithms can learn from data, generalize to unseen data, and perform tasks without explicit instructions. ML has found applications in various fields such as natural language processing, computer vision, speech recognition, and more. If you would like 

{'input': 'I want to learn about Machine Learnin',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence that focuses on the development and study of statistical algorithms. These algorithms can learn from data, generalize to unseen data, and perform tasks without explicit instructions. ML has found applications in various fields such as natural language processing, computer vision, speech recognition, and more. If you would like more detailed information, feel free to ask!'}

In [17]:
agent_executor.invoke({"input" : "Tell me about Attention is all you Need"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Attention is all you Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper authored by eight scientists working at Google, that introduced a new deep learning architecture known as the transformer based on attention mechanisms proposed by Bahdanau et al. in 2014. It is considered by some to be a founding paper for modern artificial intelligence, as transformers became the main architecture of large language models like those based on GPT. At the time, the focus of the"Attention Is All You Need" is a landmark research paper from 2017 authored by eight scientists at Google. It introduced a new deep learning architecture called the transformer, based on attention mechanisms proposed earlier. This paper is considered foundational for modern artificial intelligence, as transformers have become the main architecture for large language models like GPT.

> 

{'input': 'Tell me about Attention is all you Need',
 'output': '"Attention Is All You Need" is a landmark research paper from 2017 authored by eight scientists at Google. It introduced a new deep learning architecture called the transformer, based on attention mechanisms proposed earlier. This paper is considered foundational for modern artificial intelligence, as transformers have become the main architecture for large language models like GPT.'}